In [1]:

import tqdm
import pandas as pd
from diacritization_evaluation import util


In [3]:

train_data = pd.read_csv(
            'data/CA_MSA/train.csv',
            encoding="utf-8",
            sep='|', #config_manager.config["data_separator"],
            #nrows=,#config_manager.config["n_training_examples"],
            header=None,
        )

data = list(train_data[0])


In [4]:
d_db = {}
for line in tqdm.tqdm(data):
    for l in line.split():
        try:
            k = ''.join(util.extract_haraqat(l)[1])
            if not d_db.get(k, False):
                d_db[k] = [l]
            else:
                if not l in d_db[k]:
                    d_db[k] = sorted(d_db[k]+[l])
        except:
            pass # print('err')

100%|██████████| 2069217/2069217 [05:45<00:00, 5989.21it/s]


In [49]:

from util.constants import ARAB_CHARS, ALL_POSSIBLE_HARAQAT

d_arabic_chars_c_id = dict((c,i) for i,c in enumerate(list(ARAB_CHARS)))
d_arabic_chars_id_c = dict((i,c) for i,c in enumerate(list(ARAB_CHARS)))

d_haraqat_chars_c_id = dict((c,i) for i,c in enumerate(list(ALL_POSSIBLE_HARAQAT)))
d_haraqat_chars_id_c = dict((i,c) for i,c in enumerate(list(ALL_POSSIBLE_HARAQAT)))


def dist_haraqats_0(aa, bb):
   return sum([a==b for a,b in zip(aa, bb)]) / len(aa)

def dist_arabic_strings(str_a, str_b):
    return dist_haraqats_0(util.extract_haraqat(str_a)[2], 
                           util.extract_haraqat(str_b)[2])
 
def search(str, d_db):
    str_extract = ''.join(util.extract_haraqat(str)[1])
    if d_db.get(str_extract, False):
        ds = [{'string': s, 'score': dist_arabic_strings(str, s)} for s in d_db[str_extract]]
        return sorted(ds, key=lambda x: x['score'], reverse=True)
    else:
        return [{'string': str, 'score': -1}]


In [52]:

str = 'رِوايَةِ'
#str = data[0].split()[1]

search(str, d_db)[0]


{'string': 'رِوايَةِ', 'score': 1.0}

In [79]:

def transform_line(line):
    new_line = []
    for str in line.split():
        #print(len(str))
        if len(str) > 1:
            s = search(str, d_db)[0]
            #print(s['score'])
            new_line.append(s['string'])
            #print(str, search(str, d_db)[0]['string'])
        else:
            new_line.append(str)   
    return ' '.join(new_line)


In [95]:

f = open("arabic-text-diacritization/helpers/test_600_diacritized.txt", "r")
data = f.readlines()

data_traformed = [transform_line(d[:-1]) for d in tqdm.tqdm(data)] #[:10]

with open("arabic-text-diacritization/helpers/test_600_diacritized_searched.txt", 'w') as the_file:
    #[the_file.writelines(d) for d in data_traformed]
    for d in data_traformed:
        the_file.write(d+'\n')


100%|██████████| 2026/2026 [00:08<00:00, 231.73it/s]


In [98]:

f = open("arabic-text-diacritization/helpers/val_600_diacritized.txt", "r")
data = f.readlines()

data_traformed = [transform_line(d[:-1]) for d in tqdm.tqdm(data)]#[:10]

with open("arabic-text-diacritization/helpers/val_600_diacritized_searched.txt", 'w') as the_file:
    #[the_file.writelines(d) for d in data_traformed]
    for d in data_traformed:
        the_file.write(d+'\n')



100%|██████████| 2068/2068 [00:20<00:00, 101.88it/s]
